# Lab 3 
## Nick Bias
### 4/17/22

### Libraries 

In [1]:
# For data cleaning 
import numpy as np
import pandas as pd
import datetime

# for models 
# compare ensemble to each standalone models for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from matplotlib import pyplot

# For splitting data into training and testing 
from sklearn.model_selection import train_test_split
# example of calculate the mean absolute error
from sklearn.metrics import mean_absolute_error

### Importing and Combining the Data

In [2]:
jan = pd.read_csv("Data/Week3/January.csv")
jan = jan.rename(columns={'JANUARY': 'DoW'})
jan = jan.rename(columns={'HEART RATE UNDER RESTING': 'HEART RATE BELOW RESTING'})
# Drop first row of NAs
# by selecting all rows from first row onwards
jan = jan.iloc[1: , :]

feb = pd.read_csv("Data/Week3/February.csv")
feb = feb.rename(columns={'FEBEUARY': 'DoW'})
feb = feb.rename(columns={'SLEEP SQORE': 'SLEEP SCORE'})

nov = pd.read_csv("Data/Week3/November.csv")
nov = nov.rename(columns={'NOVEMBER': 'DoW'})
nov = nov.iloc[1: , :]

dec = pd.read_csv("Data/Week3/December.csv")
dec = dec.rename(columns={'DECEMBER': 'DoW'})
dec = dec.iloc[1: , :]

sleep_data = pd.concat([nov, dec, jan, feb])

sleep_data.rename(columns={'DATE': 'date', 'SLEEP SCORE': 'sleep_score',
                      'HOURS OF SLEEP': 'hours_sleep',
                     'REM SLEEP': 'rem_percent',
                     'DEEP SLEEP':'deep_percent',
                     'HEART RATE BELOW RESTING': 'hr_below_resting',
                     'SLEEP TIME':'sleep_time'}, inplace=True)


sleep_data

,DoW,date,sleep_score,hours_sleep,rem_percent,deep_percent,hr_below_resting,sleep_time
1,Monday,11/1/2021,88.0,8:06:00,20.00%,13.00%,84.00%,10:41pm - 7:54am
2,Tuesday,11/2/2021,83.0,7:57:00,12.00%,18.00%,90.00%,10:40pm - 7:55am
3,Wednesday,11/3/2021,81.0,7:06:00,13.00%,22.00%,93.00%,11:03pm - 7:16am
4,Thursday,11/4/2021,86.0,7:04:00,19.00%,17.00%,97.00%,10:55pm - 6:56am
5,Friday,11/5/2021,81.0,9:24:00,17.00%,15.00%,66.00%,10:14pm - 9:01am
...,...,...,...,...,...,...,...,...
23,Thursday,02/24/2022,84.0,7:18:00,19.00%,19.00%,96.00%,10:23pm - 6:43am
24,Friday,02/25/2022,90.0,7:46:00,20.00%,21.00%,98.00%,10:28pm - 7:15am
25,Saturday,02/26/2022,82.0,8:02:00,16.00%,16.00%,57.00%,11:05pm - 8:10am
26,Sunday,02/27/2022,85.0,8:54:00,18.00%,17.00%,81.00%,9:59pm - 8:11am


### Cleaning Data

In [3]:
# Splitting Sleep time into 2 seperate columns 
sleep_data[['sleep_time','wake_time']] = sleep_data["sleep_time"].str.split(" - ", n = 1, expand = True)
# Splitting hours of sleep into 3 columns 
sleep_data[['hours','minutes', 'seconds']] = sleep_data["hours_sleep"].str.split(":", n = 2, expand = True)
# Converting hours of sleep columns into float datatypes 
sleep_data = sleep_data.astype({'hours':'float', 'minutes':'float', 'seconds':'float'})

In [4]:
# Make a column of Total Minutes of Sleep
sleep_data['min_sleep'] = ((sleep_data['hours'] * 60) + sleep_data['minutes'])

# Convert Percent Columns to float 
sleep_data['rem_percent'] = sleep_data['rem_percent'].str.rstrip('%').astype('float') / 100.0
sleep_data['deep_percent'] = sleep_data['deep_percent'].str.rstrip('%').astype('float') / 100.0
sleep_data['hr_below_resting'] = sleep_data['hr_below_resting'].str.rstrip('%').astype('float') / 100.0

# Fixes 1 row were there is a - instead of : for time 
sleep_data['sleep_time'] = sleep_data['sleep_time'].astype(str)
sleep_data['sleep_time'] = sleep_data['sleep_time'].str.replace('-', ':')

In [5]:
# Convert time to 24 hour period 
sleep_data['sleep_time'] = pd.to_datetime(sleep_data['sleep_time']).dt.strftime('%H:%M:%S')
sleep_data['wake_time'] = pd.to_datetime(sleep_data['wake_time']).dt.strftime('%H:%M:%S')

sleep_data['wake_time'] = pd.to_datetime(sleep_data['wake_time'], format='%H:%M:%S')
sleep_data['sleep_time'] = pd.to_datetime(sleep_data['sleep_time'], format='%H:%M:%S')

# Find the time they spent in Bed
sleep_data['time_bed'] = sleep_data['wake_time'] - sleep_data['sleep_time']

In [6]:
# Cleaning string Results
sleep_data['time_bed'] = sleep_data['time_bed'].astype(str)
sleep_data['time_bed'] = sleep_data['time_bed'].str.replace('-1 days', '')
sleep_data['time_bed'] = sleep_data['time_bed'].str.replace('0 days', '')
sleep_data['time_bed'] = sleep_data['time_bed'].str.replace('+', '')

In [7]:
sleep_data[['bed_hours','bed_minutes', 'bed_seconds']] = sleep_data["time_bed"].str.split(":", n = 2, expand = True)
# Converting hours of sleep columns into float datatypes 
sleep_data = sleep_data.astype({'bed_hours':'float', 'bed_minutes':'float', 'bed_seconds':'float'})

# Make a column of Total Minutes of Sleep
sleep_data['minutes_in_bed'] = ((sleep_data['bed_hours'] * 60) + sleep_data['bed_minutes'])

# Calculating Percent of Night they were awake and in light sleep
sleep_data['awake_percent'] = 1 - (sleep_data['min_sleep'] / sleep_data['minutes_in_bed'])
sleep_data['light_percent'] = 1 - (sleep_data['rem_percent'] + sleep_data['deep_percent'] + sleep_data['awake_percent'])

sleep_data

,DoW,date,sleep_score,hours_sleep,rem_percent,deep_percent,hr_below_resting,sleep_time,wake_time,hours,minutes,seconds,min_sleep,time_bed,bed_hours,bed_minutes,bed_seconds,minutes_in_bed,awake_percent,light_percent
1,Monday,11/1/2021,88.0,8:06:00,0.20,0.13,0.84,1900-01-01 22:41:00,1900-01-01 07:54:00,8.0,6.0,0.0,486.0,09:13:00,9.0,13.0,0.0,553.0,0.121157,0.548843
2,Tuesday,11/2/2021,83.0,7:57:00,0.12,0.18,0.90,1900-01-01 22:40:00,1900-01-01 07:55:00,7.0,57.0,0.0,477.0,09:15:00,9.0,15.0,0.0,555.0,0.140541,0.559459
3,Wednesday,11/3/2021,81.0,7:06:00,0.13,0.22,0.93,1900-01-01 23:03:00,1900-01-01 07:16:00,7.0,6.0,0.0,426.0,08:13:00,8.0,13.0,0.0,493.0,0.135903,0.514097
4,Thursday,11/4/2021,86.0,7:04:00,0.19,0.17,0.97,1900-01-01 22:55:00,1900-01-01 06:56:00,7.0,4.0,0.0,424.0,08:01:00,8.0,1.0,0.0,481.0,0.118503,0.521497
5,Friday,11/5/2021,81.0,9:24:00,0.17,0.15,0.66,1900-01-01 22:14:00,1900-01-01 09:01:00,9.0,24.0,0.0,564.0,10:47:00,10.0,47.0,0.0,647.0,0.128284,0.551716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Thursday,02/24/2022,84.0,7:18:00,0.19,0.19,0.96,1900-01-01 22:23:00,1900-01-01 06:43:00,7.0,18.0,0.0,438.0,08:20:00,8.0,20.0,0.0,500.0,0.124000,0.496000
24,Friday,02/25/2022,90.0,7:46:00,0.20,0.21,0.98,1900-01-01 22:28:00,1900-01-01 07:15:00,7.0,46.0,0.0,466.0,08:47:00,8.0,47.0,0.0,527.0,0.115750,0.474250
25,Saturday,02/26/2022,82.0,8:02:00,0.16,0.16,0.57,1900-01-01 23:05:00,1900-01-01 08:10:00,8.0,2.0,0.0,482.0,09:05:00,9.0,5.0,0.0,545.0,0.115596,0.564404
26,Sunday,02/27/2022,85.0,8:54:00,0.18,0.17,0.81,1900-01-01 21:59:00,1900-01-01 08:11:00,8.0,54.0,0.0,534.0,10:12:00,10.0,12.0,0.0,612.0,0.127451,0.522549


In [8]:
# Selecting Variables 
sleep_data_clean = sleep_data[['sleep_score', 'DoW', 'rem_percent', 'deep_percent', 'hr_below_resting', 'min_sleep', 'minutes_in_bed', 'awake_percent', 'light_percent']]
sleep_data_clean = sleep_data_clean.reset_index(drop=True)
sleep_data_clean = sleep_data_clean.drop(5) # dropping row where math did not work out well
# Creating Dummy Variables for Days of the Week
sleep_data_DoW = pd.get_dummies(sleep_data_clean['DoW'])
sleep_data_clean = pd.merge(sleep_data_clean, sleep_data_DoW, left_index=True, right_index=True)
sleep_data_clean

,sleep_score,DoW,rem_percent,deep_percent,hr_below_resting,min_sleep,minutes_in_bed,awake_percent,light_percent,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,88.0,Monday,0.20,0.13,0.84,486.0,553.0,0.121157,0.548843,0,1,0,0,0,0,0
1,83.0,Tuesday,0.12,0.18,0.90,477.0,555.0,0.140541,0.559459,0,0,0,0,0,1,0
2,81.0,Wednesday,0.13,0.22,0.93,426.0,493.0,0.135903,0.514097,0,0,0,0,0,0,1
3,86.0,Thursday,0.19,0.17,0.97,424.0,481.0,0.118503,0.521497,0,0,0,0,1,0,0
4,81.0,Friday,0.17,0.15,0.66,564.0,647.0,0.128284,0.551716,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,84.0,Thursday,0.19,0.19,0.96,438.0,500.0,0.124000,0.496000,0,0,0,0,1,0,0
116,90.0,Friday,0.20,0.21,0.98,466.0,527.0,0.115750,0.474250,1,0,0,0,0,0,0
117,82.0,Saturday,0.16,0.16,0.57,482.0,545.0,0.115596,0.564404,0,0,1,0,0,0,0
118,85.0,Sunday,0.18,0.17,0.81,534.0,612.0,0.127451,0.522549,0,0,0,1,0,0,0


### Spliting Prediction Variable from dataset
- X = Dataset with all Independent Variables 
- y = The Dependent Variable of Sleep Score

In [9]:
y = sleep_data_clean['sleep_score']

X = sleep_data_clean.iloc[:,2:-1]
X

,rem_percent,deep_percent,hr_below_resting,min_sleep,minutes_in_bed,awake_percent,light_percent,Friday,Monday,Saturday,Sunday,Thursday,Tuesday
0,0.20,0.13,0.84,486.0,553.0,0.121157,0.548843,0,1,0,0,0,0
1,0.12,0.18,0.90,477.0,555.0,0.140541,0.559459,0,0,0,0,0,1
2,0.13,0.22,0.93,426.0,493.0,0.135903,0.514097,0,0,0,0,0,0
3,0.19,0.17,0.97,424.0,481.0,0.118503,0.521497,0,0,0,0,1,0
4,0.17,0.15,0.66,564.0,647.0,0.128284,0.551716,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.19,0.19,0.96,438.0,500.0,0.124000,0.496000,0,0,0,0,1,0
116,0.20,0.21,0.98,466.0,527.0,0.115750,0.474250,1,0,0,0,0,0
117,0.16,0.16,0.57,482.0,545.0,0.115596,0.564404,0,0,1,0,0,0
118,0.18,0.17,0.81,534.0,612.0,0.127451,0.522549,0,0,0,1,0,0


### Creating Models

In [10]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', BaggingRegressor(base_estimator=LinearRegression())))
    level0.append(('knn', BaggingRegressor(base_estimator=KNeighborsRegressor())))
    level0.append(('cart', BaggingRegressor(base_estimator=DecisionTreeRegressor())))
    level0.append(('forest', BaggingRegressor(base_estimator=RandomForestRegressor())))
    level0.append(('svm', BaggingRegressor(base_estimator=SVR())))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5, passthrough = True)
    return model
# passthrough is used for the stacking to take the og dataset instead of just the other model results

In [11]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LinearRegression()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['forest'] = RandomForestRegressor()
    models['svm'] = SVR()
    models['stacking'] = get_stacking()
    return models

In [12]:
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [13]:
# get the models to evaluate
models = get_models()

### Model Evaluation

In [14]:
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>lr -1.573 (0.319)
>knn -2.997 (0.559)
>cart -1.685 (0.502)
>forest -1.411 (0.374)
>svm -3.344 (0.614)
>stacking -1.276 (0.258)


Numbers close to 0 mean better model fit. As we can see that best model was the Stacked Model and the second best was the Randome Forest Regressor model. These numbers are very close to 0, meaning they are predicting well.

### Predictions 

In [15]:
# Splitting the Dataset into a Training and Testing set 
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [16]:
# Fitting model(s) to Training Data
models['stacking'].fit(x_train,y_train)

StackingRegressor(cv=5,
                  estimators=[('lr',
                               BaggingRegressor(base_estimator=LinearRegression())),
                              ('knn',
                               BaggingRegressor(base_estimator=KNeighborsRegressor())),
                              ('cart',
                               BaggingRegressor(base_estimator=DecisionTreeRegressor())),
                              ('forest',
                               BaggingRegressor(base_estimator=RandomForestRegressor())),
                              ('svm', BaggingRegressor(base_estimator=SVR()))],
                  final_estimator=LinearRegression(), passthrough=True)

In [17]:
yhat = model.predict(x_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print(mae)

1.158576135094285


The Baseline Stacking Model From Model Evaluations achieved a Negative Mean Absolute Error of -1.276. This Trained model was able to perform better and achieve a Mean Absolute Error of 1.159. This is a difference of 0.117, which is not a lot, but it is still better. This MAE is a measure of the average magnitude of the errors in a set of predictions, without considering their direction. Since the model achieved a score so close to 0, there are not many errors in the predictions. So this model predictions were a success. 